In [1]:
import random, os, torch
import numpy as np
from models.imagetransformer import ImageTransformer
from dataset_utils.training_dataset_creation import TrainDataset
import torch.nn as nn
import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


In [2]:
# Import 3DDFA Packages
from models.DDFA import *
import yaml
from FaceBoxes import FaceBoxes
from TDDFA import TDDFA

In [3]:
seed = 17
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [4]:
paths = []
train_dir_real = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/real/Images/RealTrainSet/real/'
train_dir_fake = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeTrainSet/FaceSwap/'
train_dir_fake_2 = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeTrainSet/Face2Face/'
train_dir_fake_3 = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeTrainSet/Deepfakes/'
train_dir_fake_4 = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeTrainSet/NeuralTextures/'


valid_dir_real = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/real/Images/RealValidationSet/real/'
valid_dir_fake = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeValidationSet/FaceSwap/'
valid_dir_fake_2 = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeValidationSet/Face2Face/'
valid_dir_fake_3 = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeValidationSet/Deepfakes/'
valid_dir_fake_4 = '/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Datasets/DeepfakeDetection/FaceForensics/fake/Images/FakeValidationSet/NeuralTextures/'

paths.append(train_dir_real)
paths.append(train_dir_fake)
paths.append(train_dir_fake_2)
paths.append(train_dir_fake_3)
paths.append(train_dir_fake_4)
paths.append(valid_dir_real)
paths.append(valid_dir_fake)
paths.append(valid_dir_fake_2)
paths.append(valid_dir_fake_3)
paths.append(valid_dir_fake_4)

In [5]:
batch_size = 6
train_loader, valid_loader = TrainDataset.get_image_batches(paths, batch_size)

Train Data Real: 106543
Train Data Fake: 36155
Train Data Fake 2: 36156
Train Data Fake 3: 36121
Train Data Fake 4: 36131
Labels: ['Face2Face', 'Deepfakes', 'FaceSwap', 'FaceSwap', 'Face2Face', 'real', 'NeuralTextures', 'FaceSwap', 'real', 'real']
200000 33334
40000 6667


In [6]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'
model = ImageTransformer('B_16_imagenet1k', pretrained=True, image_size = 300, num_classes = 2,
                        seq_embed=True, hybrid=False, device=device)

cpu
Resized positional embeddings from torch.Size([1, 577, 768]) to torch.Size([1, 325, 768])
Missing keys when loading pretrained weights: ['patch_embedding.weight', 'patch_embedding.bias', 'sequence_embedding.seq_embedding', 'linear_1.weight', 'linear_1.bias', 'proj.weight', 'proj.bias', 'fc.weight', 'fc.bias']
Unexpected keys when loading pretrained weights: []


In [7]:
epochs = 15
lr = 3e-3
# gamma = 0.7

# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# scheduler
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

In [8]:
# NO SegEmbed f2f only eval on with_new_train_valid_strategy
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for data, label in (train_loader):
        data = data.to(device)
        label = label.to(device)
        output = model(data)
        loss = criterion(output, label)
#         print("HERE")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in (valid_loader):
            data = data.to(device)
            label = label.to(device)
#             print(data.shape)aa
            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)
    
#     if epoch == 1:
#         PATH = ("/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Code/new_ensemble_vit_2epochs_fs_df_f2f_nt.pth")
#         torch.save(model.state_dict(), PATH)
#     elif epoch == 2:
#         PATH = ("/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Code/new_ensemble_vit_3epochs_fs_df_f2f_nt.pth")
#         torch.save(model.state_dict(), PATH)
#     elif epoch == 3:
#         PATH = ("/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Code/new_ensemble_vit_4epochs_fs_df_f2f_nt.pth")
#         torch.save(model.state_dict(), PATH)
#     elif epoch == 4:
#         PATH = ("/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Code/new_ensemble_vit_5epochs_fs_df_f2f_nt.pth")
#         torch.save(model.state_dict(), PATH)
#     elif epoch == 5:
#         PATH = ("/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Code/new_ensemble_vit_6epochs_fs_df_f2f_nt.pth")
#         torch.save(model.state_dict(), PATH)
#     elif epoch == 6:
#         PATH = ("/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Code/new_ensemble_vit_7epochs_fs_df_f2f_nt.pth")
#         torch.save(model.state_dict(), PATH)
#     elif epoch == 7:
#         PATH = ("/Users/Sohail/Desktop/Research/PhD/Year1/NewDeepfakeDetector/Code/new_ensemble_vit_8epochs_fs_df_f2f_nt.pth")
#         torch.save(model.state_dict(), PATH)
        
        
    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

torch.Size([1, 162, 768])
torch.Size([1, 162, 768])
torch.Size([1, 162, 768])
torch.Size([1, 162, 768])
torch.Size([1, 162, 768])
torch.Size([1, 162, 768])


KeyboardInterrupt: 

In [ ]:
for data, label in train_loader:
    print(label)
    break